In [14]:
import os
import zipfile
import requests
import arcpy
import csv

# Path to the refinery CSV data
refinery_csv = r"C:\Users\ashee\OneDrive\GIS_Files\GIS Update\Task 6 - GIS Updates\INTEK GIS MODEL\INTEK GIS Model 2017\GIS UPDATE IMPORTED SHAPEFILES\GIS Update\Refineries\Refineries.csv"
out_feature_class = r"C:\Users\ashee\OneDrive\GIS_Files\GIS Update\Task 6 - GIS Updates\INTEK GIS MODEL\INTEK GIS Model 2017\GIS UPDATE IMPORTED SHAPEFILES\GIS Update\Refineries"

# Display xy data on the map
def xy_coordinates(refinery_csv):
    # Convert CSV to point features (longitude and latitude columns)
    out_fc = os.path.join(out_feature_class, "Refineries.shp")
    arcpy.management.XYTableToPoint(refinery_csv, out_fc, "LONGITUDE", "LATITUDE")
    return out_fc

    # First, create the point feature class
# Export shapefile with proper coordinates
def export_xy_shapefile(refinery_csv):
    export_fc = xy_coordinates(refinery_csv)
    return export_fc

def source_field_alias():
    # Path to the shapefile
    shapefile = r"C:\Users\ashee\OneDrive\GIS_Files\GIS Update\Task 6 - GIS Updates\INTEK GIS MODEL\INTEK GIS Model 2017\GIS UPDATE IMPORTED SHAPEFILES\GIS Update\Refineries\Refineries.shp"
    # Path to the output geodatabase
    gdb_path = r"C:\Users\ashee\OneDrive\GIS_Files\GIS Update\Task 6 - GIS Updates\INTEK GIS MODEL\INTEK GIS Model 2017\INTEK GIS Model 2017.gdb"
    # Path to the CSV file containing the alias mappings
    csv_file_path = r"C:\GIS_Update_Alias_Automation\Alias_Keys\Refinery_Alias_Key.csv"

    # Convert the shapefile to a feature class in the geodatabase
    feature_class_name = "Refineries_fc"
    arcpy.FeatureClassToFeatureClass_conversion(shapefile, gdb_path, feature_class_name)

    # Create a dictionary to store field aliases from the CSV
    field_aliases = {}
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            field_name = row['Field']
            alias = row['Alias']
            field_aliases[field_name] = alias
            
    return field_aliases

# Function to update field aliases
def update_field_aliases(feature_class, field_aliases):
    fields = arcpy.ListFields(feature_class)
    for field in fields:
        field_name = field.name
        if field_name in field_aliases:
            field_alias = field_aliases[field_name]
            # Check field types to avoid modifying OID, Geometry, FID, and Shape
            if field.type not in ["OID", "Geometry"] and field_name not in ["FID", "Shape"]:
                arcpy.AlterField_management(feature_class, field_name, new_field_alias=field_alias)
                print(f"Changed alias for field '{field_name}' to '{field_alias}'")
            else:
                print(f"Skipping alias change for field '{field_name}' of type '{field.type}'")

    print("Field aliases updated successfully.")        
    return feature_class
        
# Path to the new feature class in the geodatabase
new_feature_class = os.path.join(gdb_path, "Refineries_fc")

# Function to apply symbology from a layer
def apply_symbology(symboLayer):
    # Path to the symbology layer
    symbology_layer = r"C:\Users\ashee\OneDrive\GIS_Files\GIS Update\Task 6 - GIS Updates\INTEK GIS MODEL\INTEK GIS Model 2017\Symbology Layers\Refineries.lyrx"
    
    try:
        # Apply symbology from the layer file to the exported layer
        arcpy.management.ApplySymbologyFromLayer(symboLayer, symbology_layer)
        print(f"Symbology applied from {symbology_layer} to {symboLayer}.")
    except Exception as e:
        print(f"Error applying symbology: {e}")
        raise  #"Refineries" Raise the exception to see the full error trace

# Run the process
exportLayer = export_xy_shapefile(refinery_csv)
field_aliases = source_field_alias()
updated_feature_class = update_field_aliases(new_feature_class, field_aliases)
apply_symbology(updated_feature_class)


Skipping alias change for field 'OBJECTID' of type 'OID'
Skipping alias change for field 'Shape' of type 'Geometry'
Changed alias for field 'LATITUDE' to 'Lat'
Changed alias for field 'LONGITUDE' to 'Long'
Changed alias for field 'Data_Vinta' to 'Data Vintage'
Changed alias for field 'PADD' to 'PADD'
Changed alias for field 'SPRRegion' to 'SPR Region'
Changed alias for field 'SiteId' to 'Site Id'
Changed alias for field 'Facility' to 'Facility'
Changed alias for field 'Facility_L' to 'Facility Location'
Changed alias for field 'Status' to 'Status'
Changed alias for field 'Notes' to 'Notes'
Changed alias for field 'Domestic' to 'Domestic '
Changed alias for field 'Throughput' to 'Throughput '
Changed alias for field 'Third_Part' to 'Third Party Imports '
Changed alias for field 'Shipments_' to 'Shipments to Other Refineries '
Changed alias for field 'ALGERIA_Li' to 'Algeria Light Sweet '
Changed alias for field 'ALGERIA_He' to 'Algeria Heavy Sweet '
Changed alias for field 'ALGERIA__1' 